In [1]:
import numpy as np
import pandas as pd

In [24]:
# finance_news = pd.read_csv('news.csv', encoding = 'ISO-8859-1')
# finance_news.head()
finance_news = pd.read_excel('mergedtitle.xlsx')
finance_news

,Unnamed: 0,Date,title,Price
0,0,20190101,Is it true that President Moon Jae-in's remark...,0
1,1,20190102,Samsung Display's investment expansion 'good n...,0
2,2,20190103,[Editorial] The KOSPI that collapsed in the 20...,1
3,3,20190104,"""Gangnam-do 2 billion won is too expensive"" Th...",1
4,4,20190105,"[Photo] BLACKPINK Jisoo, 'Chic Eyes' (Golden D...",0
...,...,...,...,...
1091,1091,20211227,"61 million won Bitcoin, 25 million won in the ...",1
1092,1092,20211228,"Ants sell Samsung Electronics 584 billion won,...",0
1093,1093,20211229,"""60% of serious accidents happen in the constr...",0
1094,1094,20211230,"Independent management of SK 'Intel NAND' ""We ...",0


In [25]:
X = finance_news['title'].to_list()
y = finance_news['Price'].to_list()

for i,x in enumerate(X):
    print(i+1,")",x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [27]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [37]:
sent_val = list()
for x in X:
    
    inputs = tokenizer(x, return_tensors="pt", padding=True)
#     outputs = finbert(inputs)
    outputs = finbert(**inputs)[0]
   
    val = labels[np.argmax(outputs.detach().numpy())]
#     print(x, '----', val)
#     print('#######################################################')    
#     sent_val.append(val)

RuntimeError: The size of tensor a (2107) must match the size of tensor b (512) at non-singleton dimension 1

In [12]:
len(sent_val)

4846

In [13]:
sent_val

['neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'negative',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'n

In [14]:
finance_news['fin_score']=''
finance_news

,sentiment,headline,fin_score
0,neutral,"According to Gran , the company has no plans t...",
1,neutral,Technopolis plans to develop in stages an area...,
2,negative,The international electronic industry company ...,
3,positive,With the new production plant the company woul...,
4,positive,According to the company 's updated strategy f...,
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,
4844,negative,Net sales of the Paper segment decreased to EU...,


In [15]:
for i in range(len(finance_news)):
    finance_news['fin_score'][i] = sent_val[i]

In [16]:
finance_news

,sentiment,headline,fin_score
0,neutral,"According to Gran , the company has no plans t...",neutral
1,neutral,Technopolis plans to develop in stages an area...,neutral
2,negative,The international electronic industry company ...,neutral
3,positive,With the new production plant the company woul...,positive
4,positive,According to the company 's updated strategy f...,positive
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,negative
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,positive
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,negative
4844,negative,Net sales of the Paper segment decreased to EU...,negative


In [8]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y, sent_val))

0.7926124638877424


In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

sentences = ["there is a shortage of capital, and we need extra financing",  
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]
results = nlp(sentences)
print(results)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

[{'label': 'negative', 'score': 0.9966173768043518}, {'label': 'positive', 'score': 1.0}, {'label': 'negative', 'score': 0.9999710321426392}, {'label': 'neutral', 'score': 0.9889442920684814}]


In [10]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ym\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

df= pd.read_csv('news.csv', encoding = 'ISO-8859-1')
df.head()

,sentiment,headline
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [12]:
results = []

for headline in df['headline']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['line'] = headline # add headlines for viewing
    results.append(pol_score)

results

[{'neg': 0.092,
  'neu': 0.837,
  'pos': 0.071,
  'compound': -0.128,
  'line': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .'},
 {'neg': 0.073,
  'neu': 0.927,
  'pos': 0.0,
  'compound': -0.296,
  'line': 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .'},
 {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0,
  'line': 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .'},
 {'neg': 0.038,
  'neu': 0.66,
  'pos': 0.302,
  'compound': 0.8555,
  'line': 'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the

In [13]:
df['Score'] = pd.DataFrame(results)['compound']
df

,sentiment,headline,Score
0,neutral,"According to Gran , the company has no plans t...",-0.1280
1,neutral,Technopolis plans to develop in stages an area...,-0.2960
2,negative,The international electronic industry company ...,0.0000
3,positive,With the new production plant the company woul...,0.8555
4,positive,According to the company 's updated strategy f...,0.6705
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,-0.7269
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,0.0000
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,0.7430
4844,negative,Net sales of the Paper segment decreased to EU...,0.4404
